In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from pandas.io.json import json_normalize
import seaborn as sb
import matplotlib.pyplot as plt
import plotly.express as px
%matplotlib inline

##load datasets to dataframes
ride1=pd.read_csv(r'E:\DS 3250\Project\bikeshare-ridership-2017\2017 Data\Bikeshare Ridership (2017 Q1).csv',sep=',')
ride2=pd.read_csv(r'E:\DS 3250\Project\bikeshare-ridership-2017\2017 Data\Bikeshare Ridership (2017 Q2).csv',sep=',')
ride3=pd.read_csv(r'E:\DS 3250\Project\bikeshare-ridership-2017\2017 Data\Bikeshare Ridership (2017 Q3).csv',sep=',')
ride4=pd.read_csv(r'E:\DS 3250\Project\bikeshare-ridership-2017\2017 Data\Bikeshare Ridership (2017 Q4).csv',sep=',')
ride=pd.concat([ride1,ride2,ride3,ride4])

w1=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_01-2017_P1H.csv',sep=',')
w2=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_02-2017_P1H.csv',sep=',')
w3=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_03-2017_P1H.csv',sep=',')
w4=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_04-2017_P1H.csv',sep=',')
w5=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_05-2017_P1H.csv',sep=',')
w6=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_06-2017_P1H.csv',sep=',')
w7=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_07-2017_P1H.csv',sep=',')
w8=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_08-2017_P1H.csv',sep=',')
w9=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_09-2017_P1H.csv',sep=',')
w10=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_10-2017_P1H.csv',sep=',')
w11=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_11-2017_P1H.csv',sep=',')
w12=pd.read_csv(r'E:\DS 3250\Project\Toronto weather 2017 hourly\en_climate_hourly_ON_6158355_12-2017_P1H.csv',sep=',')
weather=pd.concat([w1,w2,w3,w4,w5,w6,w7,w8,w9,w10,w11,w12])

station_json=pd.read_json(r'E:\DS 3250\Project\station_information.json')
station = pd.concat([pd.DataFrame(json_normalize(x)) for x in station_json['data']],ignore_index=True)

In [ ]:
## weather dataset at glance
weather.info()
weather.describe()

In [ ]:
##Temperature attribute analysis-box plot

fig=px.box(weather,y='Temp (°C)')
fig.show()

In [ ]:
##Temperature attribute analysis-histogram plot

fig=px.histogram(weather,x='Temp (°C)')
fig.show()

In [ ]:
##NA info
weather.isnull().sum()

In [ ]:
##Change weather from hourly to daily,only take temperature attribute
weather['Date/Time']=pd.to_datetime(weather['Date/Time'])
weather['Date']=weather['Date/Time'].dt.date
weather_daily=weather.groupby(['Date'],as_index=False)['Temp (°C)'].mean()
weather_daily

In [ ]:
##station dataset
station.info()
station.describe()

In [ ]:
##change data type
station['station_id']=station['station_id'].astype('int64')

In [ ]:
##NA information
station.isnull().sum()

In [ ]:
##ridership dataset at glance, the trip duration is pretty dirty.
ride.info()
ride.describe()

In [ ]:
## begore drop noisy data
ride.shape

In [ ]:
## Drop noisy data from trip_duration_seconds using + - 1.5 * interquartile. 
## Also, based on real life, any renting < 60s should be invalid data regards to trip time.
q1=ride['trip_duration_seconds'].quantile(0.25)
q3=ride['trip_duration_seconds'].quantile(0.75)
interquatile_range=q3-q1
ride=ride[~((ride['trip_duration_seconds']<(q1-1.5*interquatile_range))|(ride['trip_duration_seconds']>(q3+1.5*interquatile_range))
            |(ride['trip_duration_seconds']<60))].reset_index(drop=True)

In [ ]:
## After drop, roughly 7% data got dropped.
ride.shape


In [ ]:
##Trip duration attribute analysis-box plot
fig=px.box(ride,y='trip_duration_seconds')
fig.show()

In [ ]:
##Trip duration attribute analysis-histogram plot

fig=px.histogram(ride,x='trip_duration_seconds')
fig.show()

In [ ]:
## NA info, a lot of station_id are missing. Since the the station name in ridership table is not standard name cannot directly join with
## station table, so have to do some fussy logics.
ride.isnull().sum()

In [ ]:
##change data type
station['station_id']=station['station_id'].astype('int64')

In [ ]:
##Generate unique station information from ridership table
all_stations=ride[['from_station_id','from_station_name','trip_id']].append(ride[['to_station_id','to_station_name','trip_id']].rename(
    columns={'to_station_id':'from_station_id','to_station_name':'from_station_name'}))
unique_stations=all_stations.fillna(1).groupby(['from_station_id','from_station_name'],as_index=False)['trip_id'].count()
unique_stations.drop(unique_stations[unique_stations['from_station_name']==1].index,inplace=True)

In [ ]:
## Use fuzzywuzzy library to do fuzzy match for station name, the threshold set as 90 score, we try 80, have some mismatch.
from fuzzywuzzy import fuzz

no_ids = unique_stations[unique_stations['from_station_id']==1]
for idx, miss in no_ids.iterrows():
    max_score = 0
    
    for i, exist in station[['station_id', 'name']].iterrows():
        score = fuzz.ratio(miss['from_station_name'], exist['name'])
        
        if score > 90 and score > max_score:
            max_score = score
            no_ids.at[idx, 'from_station_id'] = exist['station_id']
    
    if max_score <= 90:
        print('Warnning: {0} station could not be matched to an existing station'.format(miss['from_station_name']))



In [ ]:
##Check the fuzzy effectiveness, show the matched results
unique_stations_cleaned=pd.concat([no_ids[no_ids['from_station_id']!=1],
        unique_stations[unique_stations['from_station_id']!=1]])
unique_stations_cleaned=unique_stations_cleaned.drop(columns=['trip_id'])
unique_stations_cleaned=unique_stations_cleaned.drop_duplicates().reset_index(drop=True)
n=unique_stations_cleaned.groupby('from_station_id')['from_station_name'].nunique()
unique_stations_cleaned[unique_stations_cleaned['from_station_id'].isin(n.index[n.gt(1)])].sort_values('from_station_id')

In [ ]:
unique_stations_cleaned

In [ ]:
##unique_stations_cleaned is the dataframe of station from ridership table,combine the lat, lon information from station table
unique_stations_combined=pd.merge(unique_stations_cleaned,station[['station_id','name','lat','lon']],
                                 how='inner',left_on='from_station_id',right_on='station_id')

In [ ]:
unique_stations_combined.rename(columns={'from_station_id':'original_station_id','from_station_name':'original_name'},inplace=True)
unique_stations_combined

In [ ]:
##Drop invalid data from trip_stop_time
ride.drop(ride[ride['trip_stop_time']== 'NULLNULL'].index,inplace=True)

In [ ]:
## Formating the data type
ride['from_station_id']=ride['from_station_id'].astype('int64',errors='ignore')
ride['to_station_id']=ride['to_station_id'].astype('int64',errors='ignore')
ride['trip_stop_time']=pd.to_datetime(ride['trip_stop_time'],dayfirst=True)
ride['trip_start_time']=pd.to_datetime(ride['trip_start_time'],dayfirst=True)

In [ ]:
ride.shape

In [ ]:
##Join with station for start from
data_join=pd.merge(ride,unique_stations_combined[['station_id','original_name','name','lat','lon']],how='left',
                   left_on='from_station_name',right_on='original_name')

In [ ]:
data_join.shape

In [ ]:
##Join with station for stop at
data_join=pd.merge(data_join,unique_stations_combined[['station_id','original_name','name','lat','lon']],how='left',
                   left_on='to_station_name',right_on='original_name')

In [ ]:
data_join.shape

In [ ]:
##Add the date of trip start
data_join['trip_start_date']=data_join['trip_start_time'].dt.date

In [ ]:
##Join with weather daily
data_join=pd.merge(data_join,weather_daily[['Date','Temp (°C)']],
                                 how='inner',left_on='trip_start_date',right_on='Date')

In [ ]:
data_join.shape

In [ ]:
##delete unnecessary column, rename columns
data_join.rename(columns={'station_id_x':'from_station_id_corr','station_id_y':'to_station_id_corr',
                          'lat_x':'from_lat','lon_x':'from_lon',
                          'lat_y':'to_lat','lon_y':'to_lon',
                         'name_x':'from_station_name_corr','name_y':'to_station_name_corr'},inplace=True)
data_join.drop(columns=['original_name_x','original_name_y','Date'],inplace=True)

In [ ]:
data_join

In [ ]:
data_join.info()

In [ ]:
data_join.isnull().sum()

In [ ]:
##create work day and month for visualization purpose
data_join['work_day'] = data_join['trip_start_time'].apply(lambda x: x.strftime('%a'))
data_join['month']=data_join['trip_start_time'].dt.month

In [ ]:
##save to csv
data_join.to_csv(r'E:\DS 3250\Project\data_join.csv',index=False)

In [ ]:
##file is two large, split into two
data_join[:700000].to_csv(r'E:\DS 3250\Project\data_join1.csv',index=False)
data_join[700000:].to_csv(r'E:\DS 3250\Project\data_join2.csv',index=False)